To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

### Unsloth

Goal: To convert `DeepSeek-R1-0528-Qwen3-8B` into a reasoning model via GRPO by using OpenR1's Math dataset.

We also use `langid` for language detection. Our main goal is to force the model to generate reasoning traces in Indonesian, and we create a reward function using `langid` to check this.

In [3]:
!pip install langid -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

from unsloth import FastLanguageModel
import torch

torch.cuda.empty_cache()

# Configuration
DATA_PATH = "/content/drive/train.csv"
OUTPUT_DIR = "/content/drive/MyDrive/grpo_math_model"
TRAINING_TEMPERATURE = 1.0  # Fixed temperature
MAX_SEQ_LENGTH = 4096
LORA_RANK = 32

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 07-22 23:46:54 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 07-22 23:46:54 [__init__.py:239] Automatically detected platform cuda.


In [8]:
device = torch.cuda.current_device()

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-0528-Qwen3-8B",
    max_seq_length = MAX_SEQ_LENGTH,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = LORA_RANK,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = LORA_RANK, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = LORA_RANK*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 701,
)

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.7.7: Fast Qwen3 patching. Transformers: 4.53.2. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: vLLM loading unsloth/deepseek-r1-0528-qwen3-8b-unsloth-bnb-4bit with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 4096. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.7 GB. Also swap space = 0 GB.


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.7.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [12]:
from collections import defaultdict
from datasets import load_dataset
# Global metrics tracking
training_metrics = defaultdict(list)
DATA_PATH = "/content/drive/MyDrive/train.csv"
dataset = load_dataset(
    "csv",
    data_files={"train": DATA_PATH},
    split="train"
)

def add_instruction(example):
    example["prompt"] = (
        "Please solve the following problem step by step:\n"
        f"Problem: {example['task'].strip()}\n\n"
        "Show your reasoning inside <thinking> tags, then provide your final numerical answer in brackets like [52].\n"
        "Format: <thinking>your reasoning here</thinking>[answer]"
    )
    return example

dataset = dataset.map(add_instruction)

# Create a lookup dictionary for faster access
dataset_lookup = {example["prompt"]: example for example in dataset}


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [13]:
import re
from fractions import Fraction

def parse_answer(s):
    """Extracts answer from various formats, returns float or None."""
    if s is None:
        return None, "none_input"

    s = str(s).strip()

    # Try different answer formats in order of preference
    patterns = [
        (r"\[(\d+(?:\.\d+)?)\]", "bracket_format"),
        (r"\\boxed\{(\d+(?:\.\d+)?)\}", "latex_boxed"),
        (r"boxed\{(\d+(?:\.\d+)?)\}", "boxed_format"),
        (r"answer is (\d+(?:\.\d+)?)", "answer_is"),
        (r"(\d+(?:\.\d+)?)(?:\s*$|\s*\n)", "number_at_end"),
    ]

    for pattern, format_type in patterns:
        match = re.search(pattern, s, re.IGNORECASE)
        if match:
            val = match.group(1).strip()
            break
    else:
        return None, "no_match_found"

    try:
        return float(val), format_type
    except ValueError:
        try:
            return float(Fraction(val)), f"{format_type}_fraction"
        except (ValueError, ZeroDivisionError):
            return None, "parse_error"

def check_format(prompts, completions, **kwargs):
    """
    Simple format checker:
    1. Must contain <thinking> tags
    2. Must have numeric answer in brackets [number]
    3. No other characters outside thinking tags and square brackets

    Returns 1.0 for correct format, 0.0 for incorrect format.
    """
    rewards = []

    for prompt, completion in zip(prompts, completions):
        if completion is None or len(completion.strip()) == 0:
            training_metrics["format_empty_completions"].append(1)
            rewards.append(0.0)
            continue

        training_metrics["format_empty_completions"].append(0)
        completion = completion.strip()

        # Check 1: Must contain <thinking> tags
        has_thinking_open = re.search(r'<thinking>', completion) is not None
        has_thinking_close = re.search(r'</thinking>', completion) is not None

        training_metrics["has_thinking_tags"].append(int(has_thinking_open and has_thinking_close))

        if not (has_thinking_open and has_thinking_close):
            rewards.append(0.0)
            continue

        # Check 2: Must have numeric answer in brackets
        bracket_match = re.search(r'\[(\d+(?:\.\d+)?)\]', completion)
        training_metrics["has_bracket_answer"].append(int(bracket_match is not None))

        if not bracket_match:
            rewards.append(0.0)
            continue

        # Check 3: Remove thinking tags and bracketed answer, check if anything else remains
        without_thinking = re.sub(r'<thinking>.*?</thinking>', '', completion, flags=re.DOTALL)
        without_answer = re.sub(r'\[\d+(?:\.\d+)?\]', '', without_thinking)

        has_extra_content = bool(without_answer.strip())
        training_metrics["has_extra_content"].append(int(has_extra_content))

        if has_extra_content:
            rewards.append(0.0)
            continue

        # All checks passed
        rewards.append(1.0)

    return rewards

def check_answer(prompts, completions, **kwargs):
    """
    Binary accuracy reward: 1.0 if answer matches exactly, 0.0 otherwise.
    Also tracks comprehensive metrics.
    """
    rewards = []

    for prompt, completion in zip(prompts, completions):
        # Get true answer from dataset lookup
        example = dataset_lookup.get(prompt)
        if example is None:
            training_metrics["dataset_lookup_failures"].append(1)
            rewards.append(0.0)
            continue

        training_metrics["dataset_lookup_failures"].append(0)

        # Track response length metrics
        completion_length = len(completion) if completion else 0
        word_count = len(completion.split()) if completion else 0
        training_metrics["completion_lengths"].append(completion_length)
        training_metrics["completion_word_counts"].append(word_count)

        # Track context usage
        prompt_length = len(prompt)
        total_length = prompt_length + completion_length
        training_metrics["prompt_lengths"].append(prompt_length)
        training_metrics["total_sequence_lengths"].append(total_length)
        training_metrics["context_utilization_ratio"].append(total_length / MAX_SEQ_LENGTH)

        # Check for potential context overflow
        context_overflow = total_length > MAX_SEQ_LENGTH * 0.95  # 95% threshold
        training_metrics["context_near_overflow"].append(int(context_overflow))

        # Parse answers and track parsing success
        true_val, true_parse_type = parse_answer(example["answer"])
        pred_val, pred_parse_type = parse_answer(completion)

        training_metrics["true_answer_parse_success"].append(int(true_val is not None))
        training_metrics["pred_answer_parse_success"].append(int(pred_val is not None))
        training_metrics["pred_parse_types"].append(pred_parse_type)

        # Track parsing errors
        if true_val is None:
            training_metrics["true_answer_parse_errors"].append(1)
            rewards.append(0.0)
            continue

        training_metrics["true_answer_parse_errors"].append(0)

        if pred_val is None:
            training_metrics["pred_answer_parse_errors"].append(1)
            rewards.append(0.0)
            continue

        training_metrics["pred_answer_parse_errors"].append(0)

        # Binary accuracy check
        is_correct = abs(pred_val - true_val) < 1e-6  # Float comparison with tolerance
        training_metrics["answer_accuracy"].append(int(is_correct))

        # Track error magnitudes for analysis (even though not used in reward)
        error_magnitude = abs(pred_val - true_val)
        training_metrics["answer_error_magnitudes"].append(error_magnitude)

        # Categorize error types
        if error_magnitude == 0:
            training_metrics["error_categories"].append("exact_match")
        elif error_magnitude < 1:
            training_metrics["error_categories"].append("small_error")
        elif error_magnitude < 10:
            training_metrics["error_categories"].append("medium_error")
        else:
            training_metrics["error_categories"].append("large_error")

        rewards.append(1.0 if is_correct else 0.0)

    return rewards


In [14]:
import numpy as np
import wandb

def log_batch_metrics():
    """Log aggregated metrics to wandb after each batch"""
    if not training_metrics:
        return

    # Aggregate metrics
    aggregated = {}

    for metric_name, values in training_metrics.items():
        if not values:
            continue

        if metric_name in ["pred_parse_types", "error_categories"]:
            # Handle categorical metrics
            from collections import Counter
            counter = Counter(values)
            for category, count in counter.items():
                aggregated[f"{metric_name}_{category}"] = count / len(values)
        else:
            # Handle numerical metrics
            values_array = np.array(values)
            aggregated[f"{metric_name}_mean"] = values_array.mean()
            aggregated[f"{metric_name}_std"] = values_array.std()
            if metric_name in ["completion_lengths", "completion_word_counts", "total_sequence_lengths"]:
                aggregated[f"{metric_name}_max"] = values_array.max()
                aggregated[f"{metric_name}_min"] = values_array.min()

    # Log to wandb
    wandb.log(aggregated)

    # Clear metrics for next batch
    training_metrics.clear()


In [17]:
from transformers import TrainerCallback

# Calculate lengths
maximum_length = max(len(row["prompt"]) for row in dataset)
max_prompt_length = maximum_length + 1
max_completion_length = MAX_SEQ_LENGTH - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    temperature = TRAINING_TEMPERATURE,  # Fixed to 1.0
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 701,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params=SamplingParams(
        temperature=TRAINING_TEMPERATURE,
        min_p=0.1,
        top_p=1.0,
        top_k=-1,
        seed=701,
        stop=[tokenizer.eos_token],
        include_stop_str_in_output=True,
    ),
    learning_rate=5e-6,
    weight_decay=0.01,
    warmup_ratio=0.1,
    optim="adamw_8bit",
    logging_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    num_generations=4,
    max_prompt_length=max_prompt_length,
    max_completion_length=max_completion_length,
    max_steps=100,
    save_steps=100,
    report_to="wandb",
    output_dir=OUTPUT_DIR,
)

class MetricsCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        log_batch_metrics()
        return control

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[check_format, check_answer],
    args=training_args,
    train_dataset=dataset,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [18]:
trainer.add_callback(MetricsCallback())
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,000 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 87,293,952 of 8,278,029,312 (1.05% trained)


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
